# Project
## 1. Introduction

In this project, we will analyse the **PlantGrowth** dataset using both t-tests and ANOVA.
The dataset contains measurements of plant `weight` across three different treatment groups:

- **ctrl** (control group)
- **trt1** (treatment group 1)
- **trt2** (treatment group 2)

We will:

1. Describe the dataset
2. Explain what a t-test is, its assumptions, and how it works
3. Perform a t-test between the two treatment groups `trt1` and `trt2`
4. Perform an ANOVA across the three groups `ctrl`, `trt1`, and `trt2`
5. Explain why ANOVA is more appropriate than multiple t-tests when comparing more than two groups

We will also provide relevant plots and references. All analysis will be completed in Python with frequent commits to this repository to demonstrate steady progress.